In [267]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import sys



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/concepts.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/__init__.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_inference_server.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/templates.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/base_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/relay.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evalua

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [268]:
train = import_data('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')
test = import_data('/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv')


In [269]:

  # Get a summary of the dataset including column types and non-null counts


In [270]:
y = train[['utility_agent1']]
X = train.drop('utility_agent1')
X = X.drop(['num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1'])

In [271]:
def drop_missing_data(df):
# Now you can safely call isnull()
    missing_percentage = X.isna().mean() * 100
# Drop columns with more than 50% missing values
    df = df.drop(columns=missing_percentage[missing_percentage > 50].index)
    return df
X = drop_missing_data(X)

UnboundLocalError: local variable 'X' referenced before assignment

In [ ]:
def drop_nonunique_values(df):
    global unique_counts
    unique_counts = X.nunique()
    print(unique_counts)
# Drop columns with only one unique value
    df = df.drop(columns=unique_counts[unique_counts == 1].index)
    return df
X = drop_nonunique_values(X)

In [ ]:
def columns_transform(X):
# Split 'agent1' into 5 parts and assign to new columns
    global ct
    X[['agent1_part1', 'agent1_part2', 'agent1_part3', 'agent1_part4', 'agent1_part5']] = X['agent1'].str.split('-', expand=True)

# Split 'agent2' into 5 parts and assign to new columns
    X[['agent2_part1', 'agent2_part2', 'agent2_part3', 'agent2_part4', 'agent2_part5']] = X['agent2'].str.split('-', expand=True)


    X = X.drop(['Id','agent1','agent2'], axis=1)

    categorical_features = X.select_dtypes(include=['object', 'category']).columns

    num_feat = X.select_dtypes('number').columns
    feature_names = X.columns

    from sklearn.preprocessing import OrdinalEncoder, StandardScaler
    cat_encoder = OrdinalEncoder()
    std = StandardScaler()
    from sklearn.compose import ColumnTransformer
    ct = ColumnTransformer([('cat', cat_encoder, categorical_features), ('std', StandardScaler(), num_feat)])
    X = ct.fit_transform(X)
    return X
X = columns_transform(X)




In [ ]:
def train_model():
    from sklearn.decomposition import PCA
    global pca, lgbm_reg
# Reduce to 100 principal components
    pca = PCA(n_components=100)
    X_pca = pca.fit_transform(X)
    X_train, X_valid, y_train, y_valid = train_test_split(X_pca, y, test_size=0.2, random_state=42)
    import lightgbm as lgb
    lgbm_reg = lgb.LGBMRegressor(
        learning_rate=0.05,
        num_leaves=50,
        n_estimators=1500,
        min_child_samples=42,
        subsample=0.8,
        colsample_bytree=0.9,
        reg_alpha=0.001,
        reg_lambda=0.02,
        min_split_gain=0.001,
        min_child_weight=0.002)        # Use GPU for faster training
    
    lgbm_reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
    y_predict = lgbm_reg.predict(X_valid)

# Evaluate the model
    from sklearn.metrics import mean_squared_error
    import numpy as np

# Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_valid, y_predict))
    print(f'Validation RMSE: {rmse:.4f}')


In [ ]:
import polars as pl  # Assuming you're using the `pl.Series` from the Polars library

counter = 0
def predict(test):
    global counter
    
    if counter == 0:
        # Train the model only the first time predict is called
        train_model()  # Ensure train_model() trains the LightGBM model (lgbm_reg)
    
    counter += 1
    
    # Step 1: Preprocess the test data
    test = drop_missing_data(test) # Drop rows or columns with missing data
    test = drop_nonunique_values(test)# Remove non-unique columns (e.g., constant features)
    test = columns_transform(test)  # Apply any additional transformations (e.g., encoding)
    
    # Step 2: Apply PCA transformation if needed (Ensure 'pca' is already fitted)
    test = pca.transform(test)  # This step assumes `pca` is a previously fitted PCA model
    
    # Step 3: Make predictions using the trained LightGBM model
    predictions = lgbm_reg.predict(test)  # lgbm_reg should be your trained LightGBM model
    
    # Step 4: Add the predictions as a new column in the submission DataFrame
    # Ensure `target` is the name of the target column
    return predictions
predict(test)

